In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import re
import sqlite3

# URL 設定
url = "https://en.wikipedia.org/wiki/Travel_and_Tourism_Development_Index"

# User-Agent 設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# データリクエスト
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # "2023 ranking" セクションの最初の <ol> タグを探す
    ranking_section = soup.find("ol")

    if ranking_section:
        # データ保存用リスト
        data = []

        # ランク、国名、スコアを抽出
        for rank, li in enumerate(ranking_section.find_all("li"), start=1):
            try:
                # 国名を <a> タグから抽出
                country = li.find("a").get_text(strip=True)

                # スコアを <li> タグのテキストから数値部分を抽出
                full_text = li.get_text(strip=True)
                score = re.search(r"(\d+\.\d+)$", full_text).group(1)  # スコアを正規表現で抽出

                # データをリストに保存
                data.append({"Rank": rank, "Country": country, "Score": float(score)})
            except AttributeError:
                print(f"データが不足しているためスキップ: {li}")

        # DataFrame を作成
        df = pd.DataFrame(data)

        # 表を出力
        print("\n== 2023年 旅行・観光開発指数 ==")
        print(tabulate(df, headers="keys", tablefmt="grid"))

        # CSV ファイルに保存
        csv_filename = "ttdi_2023.csv"
        df.to_csv(csv_filename, index=False, encoding="utf-8")
        print(f"\nデータが '{csv_filename}' ファイルに保存されました。")

        # SQLite データベースに保存
        db_filename = "ttdi_2023.db"
        conn = sqlite3.connect(db_filename)

        # テーブルを作成
        table_name = "ttdi_2023_ranking"
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Rank INTEGER PRIMARY KEY,
            Country TEXT NOT NULL,
            Score REAL NOT NULL
        );
        """
        conn.execute(create_table_query)

        # DataFrame のデータを SQLite に保存
        df.to_sql(table_name, conn, if_exists="replace", index=False)

        # データベース接続を閉じる
        conn.close()
        print(f"データが '{db_filename}' ファイルに保存されました。")
    else:
        print("2023 ranking セクションが見つかりませんでした。")
else:
    print(f"データ取得に失敗しました。HTTP ステータスコード: {response.status_code}")



== 2023년 여행 및 관광 개발 지수 ==
+----+--------+----------------------+---------+
|    |   Rank | Country              |   Score |
+====+========+======================+=========+
|  0 |      1 | USA                  |    5.24 |
+----+--------+----------------------+---------+
|  1 |      2 | Spain                |    5.18 |
+----+--------+----------------------+---------+
|  2 |      3 | Japan                |    5.09 |
+----+--------+----------------------+---------+
|  3 |      4 | France               |    5.07 |
+----+--------+----------------------+---------+
|  4 |      5 | Australia            |    5    |
+----+--------+----------------------+---------+
|  5 |      6 | Germany              |    5    |
+----+--------+----------------------+---------+
|  6 |      7 | United Kingdom       |    4.96 |
+----+--------+----------------------+---------+
|  7 |      8 | China                |    4.94 |
+----+--------+----------------------+---------+
|  8 |      9 | Italy                |    